# InstaCart Project 

## Ex4.8 Grouping data and Aggregating values - Task questions

### Contents
- 1. Import libraries and data frames
- 2. Group df by 'department_id' and aggregate mean of the 'order_number'
- 3.  Comparing order_number averages between full dataset and 1 million rows subset
- 4. Creating a loyalty flag for the existing customers
- 5. Find basic statistics of product prices for each loyalty category
- 6. Data Quality Issues - correcting prices in df
- 7. Repeat Q5 - basic statistics of Loyal Customer Flag - 8. Derive new column for 'Low/High Spender'
- 9. Regularity of customer orders
- 10. Export df

# 01 Import libraries and data frames
### Q1


In [1]:
# Import libraries
import pandas as pd
import numpy as np
import os

In [2]:
# Create file path
path = r'//Users/amypalomino/Documents/(03.22) InstaCart Basket Analysis/02 Data'

In [3]:
# Import dataframe ords_prods_merged
ords_prods_merge = pd.read_pickle(
            os.path.join(path,'Prepared Data', 'orders_products_merged.pkl'))

In [4]:
ords_prods_merge.shape

(32435059, 19)

In [5]:
ords_prods_merge.head()

,order_id,user_id,order_number,order_day_of_week,order_hour_of_day,days_since_prior_order,product_id,add_to_cart_order,reordered,_merge,product_name,aisle_id,department_id,prices,price_range_loc,busiest day,busiest_period_of_day,max_order,loyalty_flag
0,2539329,1,1,2,8,NaN,196,1,0,both,Soda,77.0,7.0,9.0,Mid range product,Regular Busy,Average orders,10,New customer
1,2539329,1,1,2,8,NaN,14084,2,0,both,Organic Unsweetened Vanilla Almond Milk,91.0,16.0,12.5,Mid range product,Regular Busy,Average orders,10,New customer
2,2539329,1,1,2,8,NaN,12427,3,0,both,Original Beef Jerky,23.0,19.0,4.4,Low range product,Regular Busy,Average orders,10,New customer
3,2539329,1,1,2,8,NaN,26088,4,0,both,Aged White Cheddar Popcorn,23.0,19.0,4.7,Low range product,Regular Busy,Average orders,10,New customer
4,2539329,1,1,2,8,NaN,26405,5,0,both,XL Pick-A-Size Paper Towel Rolls,54.0,17.0,1.0,Low range product,Regular Busy,Average orders,10,New customer


# 02 Group df by 'department_id' and aggregate mean of the 'order_number'
### Q 2


In [6]:
# Group df by 'department_id' and aggregate mean of the 'order_number'
ords_prods_merge.groupby('department_id').agg({'order_number' : ['mean']})

,order_number
,mean
department_id,
1.0,15.457838
2.0,17.277920
3.0,17.170395
4.0,17.811403
5.0,15.215751
6.0,16.439806
7.0,17.225802
8.0,15.340650


# 03 
### Q3 Comparing order_number averages between full dataset and 1 million rows subset
- The average value per department id isn't vastly different between the 2 data frames. A couple of the department id's show a larger variance, such as 21 which had an average of 21.99 average orders within the subset but increased to 22.9 when aggregated on the full data frame. 
- Other department ids showed a smaller average when calculated on the entire data frame, such as id 20, which reduced from 17.1 on the subset to a 16.5 average on the full data set. 
- The data frame is ordered based on user_id which implies the data frame is in chronological order. From this assumption, I can then derive another assumption, that sales within department id 21 are increasing with time, but declining over time for products sold in deprtment id 20 based on the average order numbers per department.

# 04 
### Q4 Creating a loyalty flag for the existing customers

In [7]:
ords_prods_merge['loyalty_flag'].value_counts (dropna = False)

Regular customer    15891507
Loyal customer      10294027
New customer         6249525
Name: loyalty_flag, dtype: int64

# 05 
### Q5 Find basic statistics of product prices for each loyalty category 

In [8]:
ords_prods_merge.groupby ('loyalty_flag').agg({'prices' : ['min', 'max', 'mean']})

prices                    
                    min      max       mean
loyalty_flag                               
Loyal customer      1.0  99999.0  10.386336
New customer        1.0  99999.0  13.294670
Regular customer    1.0  99999.0  12.495717

- This is interesting and not what I expected. The new customers purchase higher average price value items than the regular or even loyal customers. 
- I need to fix the incorrect price currently stored for the 2% milk (9999.0) as this will be skewing the mean  considerably



# 06 - Data Quality Issues - correcting prices in df

In [9]:
# Setting correct price for 2 % milk which is incorrectly priced at 99999.0
ords_prods_merge.loc[ords_prods_merge['prices'] == 99999.0, 'prices'] = 0.99

In [12]:
# Is there another inccorect price that needs changing?
ords_prods_merge['prices'].max()

14900.0

In [17]:
# Create subset to identify number of rows containing inccorect price of 14900.0
price = ords_prods_merge[ords_prods_merge['prices'] == 14900.0]


In [21]:
# How many rows contain the 14900 price?
price['prices'].value_counts (dropna = False)

14900.0    4429
Name: prices, dtype: int64

In [22]:
# Check product name for item priced at 14900.0
print(price)['prices','product_name']

          order_id  user_id  order_number  order_day_of_week  \
1576        912404       17            12                  2   
1638        603376       17            22                  6   
16534      3264360      135             2                  2   
16540       892534      135             3                  0   
53712       229704      342             8                  1   
...            ...      ...           ...                ...   
32350634   3172853   205650            18                  1   
32377867   2504315   205818             3                  5   
32377877   1108388   205818             5                  4   
32410707   1916142   206049             1                  2   
32410731    379732   206049             4                  1   

          order_hour_of_day  days_since_prior_order  product_id  \
1576                     14                     5.0       21553   
1638                     16                     4.0       21553   
16534                    21   

TypeError: 'NoneType' object is not subscriptable

In [34]:
# Cottage cheese is definitely not 14900.0/unit. Correcting price to 1.49
ords_prods_merge.loc[ords_prods_merge['prices'] == 14900.0, 'prices'] = 1.49

In [24]:
# Look at max price in 'prices' again
ords_prods_merge['prices'].max()

25.0

# 07 
Repeat Q5 - basic statistics of Loyal Customer Flag

In [25]:
ords_prods_merge.groupby ('loyalty_flag').agg({'prices' : ['min', 'max', 'mean']})

prices                
                    min   max      mean
loyalty_flag                           
Loyal customer     0.99  25.0  7.772758
New customer       0.99  25.0  7.800029
Regular customer   0.99  25.0  7.797197

### Average price of product across customer type
I'm suprised again, this time by how close the average is across the 3 customer types after removing the 2 outlier prices that were skewing the results

In [26]:
# Want to check that the average price across the various departments looks as I would expect 
ords_prods_merge.groupby ('department_id').agg({'prices' : ['min', 'max', 'mean']})

prices                 
                 min   max       mean
department_id                        
1.0             1.00  15.0   7.736553
2.0             1.10  15.0   6.990934
3.0             1.00  15.0   7.853809
4.0             1.00  15.0   7.981708
5.0             1.00  15.0   8.143701
6.0             1.00  15.0   7.682264
7.0             1.00  15.0   7.680502
8.0             1.00  15.0   7.889070
9.0             1.00  15.0   7.350283
10.0            1.40  14.1   8.348960
11.0            1.00  15.0   7.996769
12.0            8.00  25.0  16.294443
13.0            1.00  20.0   8.009443
14.0            1.00  14.9   8.028963
15.0            1.00  15.0   7.539976
16.0            0.99  15.0   8.328491
17.0            1.00  15.0   7.384058
18.0            1.00  15.0   7.638380
19.0            1.60   7.0   4.275437
20.0            1.00  15.0   7.778701
21.0            1.00  15.0   8.660716

# 08
### Q6 - Derive new column for 'Low/High Spender'


In [29]:
# Aggregate average price per user as 'average_price' 
ords_prods_merge['average_price'] = ords_prods_merge.groupby(['user_id'])['prices'].transform(np.mean)

In [30]:
# Assign 'Low Spender' flag to customers with an average_price below 10
ords_prods_merge.loc[ords_prods_merge['average_price'] <10,
                    'High/Low_Spender'] = 'Low Spender'

In [31]:
# Assign 'High Spender' flag to customers with an average_price equal to or above 10
ords_prods_merge.loc[ords_prods_merge['average_price'] >=10,
                    'High/Low_Spender'] = 'High Spender'

In [33]:
ords_prods_merge.head(60)

,order_id,user_id,order_number,order_day_of_week,order_hour_of_day,days_since_prior_order,product_id,add_to_cart_order,reordered,_merge,...,aisle_id,department_id,prices,price_range_loc,busiest day,busiest_period_of_day,max_order,loyalty_flag,average_price,High/Low_Spender
0,2539329,1,1,2,8,NaN,196,1,0,both,...,77.0,7.0,9.0,Mid range product,Regular Busy,Average orders,10,New customer,6.367797,Low Spender
1,2539329,1,1,2,8,NaN,14084,2,0,both,...,91.0,16.0,12.5,Mid range product,Regular Busy,Average orders,10,New customer,6.367797,Low Spender
2,2539329,1,1,2,8,NaN,12427,3,0,both,...,23.0,19.0,4.4,Low range product,Regular Busy,Average orders,10,New customer,6.367797,Low Spender
3,2539329,1,1,2,8,NaN,26088,4,0,both,...,23.0,19.0,4.7,Low range product,Regular Busy,Average orders,10,New customer,6.367797,Low Spender
4,2539329,1,1,2,8,NaN,26405,5,0,both,...,54.0,17.0,1.0,Low range product,Regular Busy,Average orders,10,New customer,6.367797,Low Spender
5,2398795,1,2,3,7,15.0,196,1,1,both,...,77.0,7.0,9.0,Mid range product,Slowest Days,Fewest orders,10,New customer,6.367797,Low Spender
6,2398795,1,2,3,7,15.0,10258,2,0,both,...,117.0,19.0,3.0,Low range product,Slowest Days,Fewest orders,10,New customer,6.367797,Low Spender
7,2398795,1,2,3,7,15.0,12427,3,1,both,...,23.0,19.0,4.4,Low range product,Slowest Days,Fewest orders,10,New customer,6.367797,Low Spender
8,2398795,1,2,3,7,15.0,13176,4,0,both,...,24.0,4.0,10.3,Mid range product,Slowest Days,Fewest orders,10,New customer,6.367797,Low Spender
9,2398795,1,2,3,7,15.0,26088,5,1,both,...,23.0,19.0,4.7,Low range product,Slowest Days,Fewest orders,10,New customer,6.367797,Low Spender


In [35]:
# Summary of high/low spending customer type
ords_prods_merge['High/Low_Spender'].value_counts (dropna = False)

Low Spender     32315213
High Spender      119846
Name: High/Low_Spender, dtype: int64

# 09. 
### Q7 Regularity of customer orders

In [36]:
# Find the median of the 'days_since_prior_order' grouped by user_id
ords_prods_merge['median_order_frequency'] = ords_prods_merge.groupby(['user_id'])['days_since_prior_order'].transform(np.median)

In [37]:
# Assign 'Non-frequent' flag to customers with a median_order_frequency above 20
ords_prods_merge.loc[ords_prods_merge['median_order_frequency'] >20,
                    'Order_frequency_flag'] = 'Non-frequent customer'

In [38]:
# Assign 'Regular csutomer' flag to customers with a median_order_frequency between 20 and 10
ords_prods_merge.loc[(ords_prods_merge['median_order_frequency'] <=20) & 
                     (ords_prods_merge['median_order_frequency'] >10),
                    'Order_frequency_flag'] = 'Regular customer'

In [39]:
# Assign 'Frequent customer' flag to customers with a median_order_frequency below 10
ords_prods_merge.loc[ords_prods_merge['median_order_frequency'] <=10,
                    'Order_frequency_flag'] = 'Frequent customer'

In [41]:
# Summary of 3 types of order_frequency customer
ords_prods_merge['Order_frequency_flag'].value_counts (dropna = False)

Frequent customer        21578273
Regular customer          7217066
Non-frequent customer     3639720
Name: Order_frequency_flag, dtype: int64

# 10. Export df

In [42]:
ords_prods_merge.to_pickle (os.path.join (path,'Prepared Data', 'orders_products_merged.pkl'))